# Конструирование признаков (Feature Engineering)

---

**Источники:**

[Искусство Feature Engineering в машинном обучении](https://habr.com/ru/company/mlclass/blog/248129/)

[Конструирование признаков](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2)

[Representation: Feature Engineering](https://developers.google.com/machine-learning/crash-course/representation/feature-engineering)

[7 Feature Engineering Techniques in Machine Learning You Should Know](https://www.analyticsvidhya.com/blog/2020/10/7-feature-engineering-techniques-machine-learning/)

[Feature Engineering — deep dive into Encoding and Binning techniques](https://towardsdatascience.com/feature-engineering-deep-dive-into-encoding-and-binning-techniques-5618d55a6b38)

[Feature Engineering in Machine Learning](https://lucasxlu.github.io/blog/2018/08/20/ml-feml/)

[Binning Data with Pandas qcut and cut](https://pbpython.com/pandas-qcut-cut.html)

[Data Preprocessing with Python Pandas — Part 5 Binning](https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950)

---

## Подготовка окружения

In [ ]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [ ]:
# !conda install matplotlib numpy missingno scikit-learn -y

In [ ]:
# !conda install -c conda-forge category_encoders -y

In [4]:
import numpy as np

np.__version__

'1.19.2'

In [5]:
import pandas as pd

pd.__version__

'1.2.3'

## Конструирование признаков (Feature Engineering)

<blockquote>Придумывать признаки трудно, требует много времени и глубоких знаний. «Прикладное машинное обучение», в основном, это конструирование признаков.
— Эндрю Ын </blockquote>

**Признак** — это качество или свойство, присущее всем независимым объектам, анализ или прогноз для которых производится. Любое качество может быть полезным, если оно полезно для модели.

Смысл **признака** и чем он отличается от качества, легче было бы понять в контексте задачи. **Признак является характеристикой, которая может помочь решить задачу**.

**Признаки** в данных важны для используемых прогнозных моделей и влияют на результат, который необходимо получить. **Качество и количество признаков оказывают большое влияние на качество модели, будет она хорошей или нет.**

Можно сказать, что чем лучше **признаки**, тем лучше будет результат. Это не совсем полная правда, поскольку результат будет зависеть от модели и данных, не просто от выбранных **признаков**. Но отбор правильных **признаков** остаётся очень важным делом. Более качественные **признаки** могут дать более простую и более гибкую модель и часто они дают лучшие результаты.

**Конструирование признаков** — это процесс использования предметной области данных для создания признаков, которые нужны для машинного обучения. 

**Конструирование признаков** является фундаментом для приложений машинного обучения, а также трудным и затратным процессом. Необходимости ручного конструирования признаков можно избежать при автоматизации прикладного обучения признакам.

Почти любая задача начинается с создания (Engineering) и отбора (Selection) признаков.

**Процесс конструирования признаков** — это
- Метод мозгового штурма или проверка признаков;
- Решение, какие признаки создавать;
- Создание признаков;
- Проверка, какие признаки работают с моделью;
- Улучшение признаков, если требуется;
- Возврат к методу мозгового штурма/создание других признаков, пока работа не будет завершена.


## Категориальные признаки

**Пример:**
Цвет (color), т.е. синий (blue), красный (red), зеленый (green).


**Возможное решение:**
- Добавить признаки вида is_red, is_blue, is_green, is_red_or_blue и другие возможные комбинации.
- Разбиение на интервалы (Binning).
- Категория -> число (`LabelEncoder` или `OneHotEncoder`).
- Заменить признаки на их количество (`CountEncoder`).
- `TargetEncoder`.
- `CatBoostEncoder`.


<img src="images/feature_engineering.png">

### Загрузка данных

[Источник (OC2Emission)](https://www.kaggle.com/gangliu/oc2emission)

In [6]:
df = pd.read_csv("./../../data/FuelConsumptionCo2.csv")
df

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1063,2014,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264
1064,2014,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1065,2014,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260


In [7]:
df_num = df.select_dtypes(include=[np.number])
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   ENGINESIZE                1067 non-null   float64
 2   CYLINDERS                 1067 non-null   int64  
 3   FUELCONSUMPTION_CITY      1067 non-null   float64
 4   FUELCONSUMPTION_HWY       1067 non-null   float64
 5   FUELCONSUMPTION_COMB      1067 non-null   float64
 6   FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 7   CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4)
memory usage: 66.8 KB


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   MAKE                      1067 non-null   object 
 2   MODEL                     1067 non-null   object 
 3   VEHICLECLASS              1067 non-null   object 
 4   ENGINESIZE                1067 non-null   float64
 5   CYLINDERS                 1067 non-null   int64  
 6   TRANSMISSION              1067 non-null   object 
 7   FUELTYPE                  1067 non-null   object 
 8   FUELCONSUMPTION_CITY      1067 non-null   float64
 9   FUELCONSUMPTION_HWY       1067 non-null   float64
 10  FUELCONSUMPTION_COMB      1067 non-null   float64
 11  FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 12  CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 108.5+ KB


In [34]:
df.MAKE.value_counts()

FORD             90
CHEVROLET        86
BMW              64
MERCEDES-BENZ    59
TOYOTA           49
AUDI             49
GMC              49
PORSCHE          44
VOLKSWAGEN       42
DODGE            39
MINI             36
NISSAN           33
KIA              33
CADILLAC         32
JEEP             31
MAZDA            27
HYUNDAI          24
SUBARU           23
LEXUS            22
JAGUAR           22
HONDA            21
INFINITI         21
CHRYSLER         19
LAND ROVER       19
MITSUBISHI       16
BUICK            16
RAM              13
ACURA            12
LINCOLN          11
VOLVO            11
FIAT             10
SCION             9
BENTLEY           8
ROLLS-ROYCE       7
ASTON MARTIN      7
MASERATI          6
LAMBORGHINI       3
SMART             2
SRT               2
Name: MAKE, dtype: int64

In [32]:
# очень много вариантов, все почти уникальные
df.MODEL.value_counts()

F150 FFV                      8
F150 FFV 4X4                  8
ACCORD                        6
FOCUS FFV                     6
BEETLE                        6
                             ..
TTS COUPE QUATTRO             1
SL 65 AMG                     1
E 350 CABRIOLET               1
CONTINENTAL GT CONVERTIBLE    1
640i xDRIVE GRAN COUPE        1
Name: MODEL, Length: 663, dtype: int64

In [35]:
df.VEHICLECLASS.value_counts()

MID-SIZE                    178
COMPACT                     172
SUV - SMALL                 154
SUV - STANDARD              110
FULL-SIZE                    86
TWO-SEATER                   71
SUBCOMPACT                   65
PICKUP TRUCK - STANDARD      62
MINICOMPACT                  47
STATION WAGON - SMALL        36
VAN - PASSENGER              25
VAN - CARGO                  22
MINIVAN                      14
PICKUP TRUCK - SMALL         12
SPECIAL PURPOSE VEHICLE       7
STATION WAGON - MID-SIZE      6
Name: VEHICLECLASS, dtype: int64

In [36]:
df.FUELTYPE.value_counts()

X    514
Z    434
E     92
D     27
Name: FUELTYPE, dtype: int64

In [14]:
cat_features = ['FUELTYPE']    # ['FUELTYPE', 'MAKE', 'VEHICLECLASS']

### Replace

In [37]:
# ВНИМАНИЕ: не удаляет значения, которые не указаны в replace
df['FUELTYPE_new'] = df.FUELTYPE.replace({'D': 1, 'E': 2, 'Z': 3, 'X': 4})
df.FUELTYPE_new.value_counts()

4    514
3    434
2     92
1     27
Name: FUELTYPE_new, dtype: int64

### Map

### OrdinalEncoder

### LabelEncoder

Объект `LabelEncoder` присваивает каждому уникальному значению отдельное целое число.

<img src="images/label_encoder_example.png"/>

Этот подход **предполагает упорядочение категорий**:

`Никогда (0) < Редко (1) < Большинство дней (2) < Каждый день (3)`.

Это предположение имеет смысл в этом примере, потому что существует неоспоримое ранжирование категорий.

Не все категориальные переменные имеют четкий порядок значений, но мы называем те, которые имеют **порядковыми переменными**.

Для моделей на основе деревьев (таких, как деревья решений = `decision trees` и случайные леса = `random forests`) можно ожидать, что кодирование меток будет хорошо работать с порядковыми переменными.

In [15]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoded = df[cat_features].apply(encoder.fit_transform)
encoded

,FUELTYPE
0,3
1,3
2,3
3,3
4,3
...,...
1062,2
1063,2
1064,2
1065,2


In [16]:
new_df = df_num.join(encoded)
new_df

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPE
0,2014,2.0,4,9.9,6.7,8.5,33,196,3
1,2014,2.4,4,11.2,7.7,9.6,29,221,3
2,2014,1.5,4,6.0,5.8,5.9,48,136,3
3,2014,3.5,6,12.7,9.1,11.1,25,255,3
4,2014,3.5,6,12.1,8.7,10.6,27,244,3
...,...,...,...,...,...,...,...,...,...
1062,2014,3.0,6,13.4,9.8,11.8,24,271,2
1063,2014,3.2,6,13.2,9.5,11.5,25,264,2
1064,2014,3.0,6,13.4,9.8,11.8,24,271,2
1065,2014,3.2,6,12.9,9.3,11.3,25,260,2


### OneHotEncoder

При использовании `OneHotEncoder` создаются новые столбцы, указывающие на наличие (или отсутствие) каждого возможного значения в исходных данных.

<img src="images/one_hot_encoder_example.png" />

В исходном наборе данных "Цвет" - это категориальная переменная с тремя категориями: "Красный", "Желтый" и "Зеленый".

Соответствующий `OneHotEncoder` содержит один столбец для каждого возможного значения и одну строку для каждой строки в исходном наборе данных.

Если исходным значением было "Красный", помещаем 1 в столбец "Красный"; если исходным значением было "Желтый", мы помещаем 1 в столбец "Желтый" и так далее.

В отличие от `LabelEncoder`, `OneHotEncoder` **не предполагает упорядочивания категорий**.

Таким образом, вы можете ожидать, что этот подход будет работать особенно хорошо, если в категориальных данных *нет четкого упорядочения* (например, "Красный" не больше и не меньше, чем "Желтый").

Категориальные переменные без внутреннего ранжирования называют **номинальными переменными**.

Объект `OneHotEncoder` обычно работает не очень хорошо, если категориальная переменная принимает большое количество значений (т.е. обычно не следует использовать его для переменных, принимающих более 15 различных значений).

In [17]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_encoded = OH_encoder.fit_transform(df[cat_features])
OH_encoded

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [18]:
column_names = OH_encoder.get_feature_names(cat_features)
column_names

array(['FUELTYPE_D', 'FUELTYPE_E', 'FUELTYPE_X', 'FUELTYPE_Z'],
      dtype=object)

In [19]:
OH_encoded_with_names = pd.DataFrame(OH_encoded, columns=column_names)
OH_encoded_with_names

,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
1062,0.0,0.0,1.0,0.0
1063,0.0,0.0,1.0,0.0
1064,0.0,0.0,1.0,0.0
1065,0.0,0.0,1.0,0.0


In [20]:
new_df = df_num.join(OH_encoded_with_names)
new_df

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,2014,2.0,4,9.9,6.7,8.5,33,196,0.0,0.0,0.0,1.0
1,2014,2.4,4,11.2,7.7,9.6,29,221,0.0,0.0,0.0,1.0
2,2014,1.5,4,6.0,5.8,5.9,48,136,0.0,0.0,0.0,1.0
3,2014,3.5,6,12.7,9.1,11.1,25,255,0.0,0.0,0.0,1.0
4,2014,3.5,6,12.1,8.7,10.6,27,244,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,3.0,6,13.4,9.8,11.8,24,271,0.0,0.0,1.0,0.0
1063,2014,3.2,6,13.2,9.5,11.5,25,264,0.0,0.0,1.0,0.0
1064,2014,3.0,6,13.4,9.8,11.8,24,271,0.0,0.0,1.0,0.0
1065,2014,3.2,6,12.9,9.3,11.3,25,260,0.0,0.0,1.0,0.0


### Dummies

- `pd.get_dummies` приводит к матрице `Pandas DataFrame`, тогда как `OneHotEncoder` приводит к матрице `SciPy CSR`.

- `pd.get_dummies` намного быстрее, чем `OneHotEncoder`.

- `OneHotEncoder` не может обрабатывать строковые значения напрямую. Если ваши номинальные характеристики представляют собой строки, вам необходимо сначала сопоставить их с целыми числами.

- `pandas.get_dummies` - полная противоположность. По умолчанию он преобразует только строковые столбцы в one-hot представление, если столбцы не указаны.

- Суть в том, что кодировщик `OneHotEncoder` создает функцию, которая сохраняется и затем может применяться к новым наборам данных, которые используют те же категориальные переменные, с согласованными результатами.

- С другой стороны, с `sklearn.OneHotEncoder`, после того как мы создали кодировщик, мы можем повторно использовать его для получения одного и того же вывода каждый раз, со столбцами только для "красного" и "зеленого".
Невозможно явно контролировать, что происходит, когда встречается новый "синий": если предполагается, что это невозможно, то можно указать, чтобы он выдал ошибку с помощью `handle_unknown = "error"`; в противном случае мы можем сказать ему продолжить и просто установить красный и зеленый столбцы в 0 с помощью `handle_unknown = "ignore"`.

In [21]:
dummies = pd.get_dummies(df[cat_features])
dummies

,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
1062,0,0,1,0
1063,0,0,1,0
1064,0,0,1,0
1065,0,0,1,0


In [22]:
new_df = df_num.join(dummies)
new_df

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPE_D,FUELTYPE_E,FUELTYPE_X,FUELTYPE_Z
0,2014,2.0,4,9.9,6.7,8.5,33,196,0,0,0,1
1,2014,2.4,4,11.2,7.7,9.6,29,221,0,0,0,1
2,2014,1.5,4,6.0,5.8,5.9,48,136,0,0,0,1
3,2014,3.5,6,12.7,9.1,11.1,25,255,0,0,0,1
4,2014,3.5,6,12.1,8.7,10.6,27,244,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,3.0,6,13.4,9.8,11.8,24,271,0,0,1,0
1063,2014,3.2,6,13.2,9.5,11.5,25,264,0,0,1,0
1064,2014,3.0,6,13.4,9.8,11.8,24,271,0,0,1,0
1065,2014,3.2,6,12.9,9.3,11.3,25,260,0,0,1,0


### CountEncoder

Заменяет каждое категориальное значение количеством раз, сколько оно появляется в наборе данных.

Необходимо установить пакет [Category Encoders](http://contrib.scikit-learn.org/category_encoders/)

In [23]:
!pip -V

pip 21.0.1 from /home/ira/.local/lib/python3.8/site-packages/pip (python 3.8)


In [24]:
!conda install -c conda-forge category_encoders -y

Solving environment: done

# All requested packages already installed.



In [25]:
import category_encoders as ce

count_enc = ce.CountEncoder()

count_encoded = count_enc.fit_transform(df[cat_features])
count_encoded

,FUELTYPE
0,434
1,434
2,434
3,434
4,434
...,...
1062,514
1063,514
1064,514
1065,514


In [26]:
new_df = df_num.join(count_encoded.add_suffix("_count"))
new_df

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPE_count
0,2014,2.0,4,9.9,6.7,8.5,33,196,434
1,2014,2.4,4,11.2,7.7,9.6,29,221,434
2,2014,1.5,4,6.0,5.8,5.9,48,136,434
3,2014,3.5,6,12.7,9.1,11.1,25,255,434
4,2014,3.5,6,12.1,8.7,10.6,27,244,434
...,...,...,...,...,...,...,...,...,...
1062,2014,3.0,6,13.4,9.8,11.8,24,271,514
1063,2014,3.2,6,13.2,9.5,11.5,25,264,514
1064,2014,3.0,6,13.4,9.8,11.8,24,271,514
1065,2014,3.2,6,12.9,9.3,11.3,25,260,514


### TargetEncoder

Заменяет категориальное значение средним значением для этого признака.

Например, учитывая дано значение для страны "RU", вычислите средний доход для всех строк с country == "RU", допустим 0.28.

Необходимо установить пакет [Category Encoders](http://contrib.scikit-learn.org/category_encoders/)

In [27]:
import category_encoders as ce

target_enc = ce.TargetEncoder(cols=cat_features)

train_df = df.copy()

target_enc = target_enc.fit_transform(train_df[cat_features], train_df['CO2EMISSIONS'])
target_enc

/home/ira/anaconda3/envs/LevelUp_DataScience/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,FUELTYPE
0,268.529954
1,268.529954
2,268.529954
3,268.529954
4,268.529954
...,...
1062,241.097276
1063,241.097276
1064,241.097276
1065,241.097276


In [28]:
new_df = df_num.join(target_enc.add_suffix("_target"))
new_df

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPE_target
0,2014,2.0,4,9.9,6.7,8.5,33,196,268.529954
1,2014,2.4,4,11.2,7.7,9.6,29,221,268.529954
2,2014,1.5,4,6.0,5.8,5.9,48,136,268.529954
3,2014,3.5,6,12.7,9.1,11.1,25,255,268.529954
4,2014,3.5,6,12.1,8.7,10.6,27,244,268.529954
...,...,...,...,...,...,...,...,...,...
1062,2014,3.0,6,13.4,9.8,11.8,24,271,241.097276
1063,2014,3.2,6,13.2,9.5,11.5,25,264,241.097276
1064,2014,3.0,6,13.4,9.8,11.8,24,271,241.097276
1065,2014,3.2,6,12.9,9.3,11.3,25,260,241.097276


### CatBoostEncoder

Похож на `TargetEncoder` в том смысле, что оно основано на целевой вероятности (target probability) для данного значения. Однако с `CatBoostEncoder` для каждой строки целевая вероятность вычисляется только из строк перед ней.

Необходимо установить пакет [Category Encoders](http://contrib.scikit-learn.org/category_encoders/)

In [29]:
import category_encoders as ce

target_enc = ce.CatBoostEncoder(cols=cat_features)

train_df = df.copy()

target_enc = target_enc.fit_transform(train_df[cat_features], train_df['CO2EMISSIONS'])
target_enc

/home/ira/anaconda3/envs/LevelUp_DataScience/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,FUELTYPE
0,256.228679
1,226.114339
2,224.409560
3,202.307170
4,212.845736
...,...
1062,240.823978
1063,240.883031
1064,240.928181
1065,240.986801


In [30]:
new_df = df_num.join(target_enc.add_suffix("_cb"))
new_df

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPE_cb
0,2014,2.0,4,9.9,6.7,8.5,33,196,256.228679
1,2014,2.4,4,11.2,7.7,9.6,29,221,226.114339
2,2014,1.5,4,6.0,5.8,5.9,48,136,224.409560
3,2014,3.5,6,12.7,9.1,11.1,25,255,202.307170
4,2014,3.5,6,12.1,8.7,10.6,27,244,212.845736
...,...,...,...,...,...,...,...,...,...
1062,2014,3.0,6,13.4,9.8,11.8,24,271,240.823978
1063,2014,3.2,6,13.2,9.5,11.5,25,264,240.883031
1064,2014,3.0,6,13.4,9.8,11.8,24,271,240.928181
1065,2014,3.2,6,12.9,9.3,11.3,25,260,240.986801


### Binning

## Числовые признаки

**Возможное решение:**
- Округление или разделение на целую и вещественную часть (+ нормализация).
- Приведение числового признака в категориальный (Binning). Пример: добавить признаки вида "рост больше X", "рост меньше X".

<img src="images/feature_engineering.png">

## Дата и время

**Возможное решение:**
- Добавить признаки, соответствующие 
    - времени дня
    - количеству прошедшего времени с определенного момента
    - выделение сезонов, времен года, кварталов, праздничных дней
    - состояние погоды для этих дней (может быть в дождливые дни поведение покупателей меняется?)
- Разделение времени на часы, минуты и секунды (если время дано в Unix-Time или ISO формате). 

## Строковые признаки

**Возможное решение:**
- В самих строках зачастую содержится информация ("Mr.", "Mrs." преобразовать в половой признак).
- Сократить количество "категорий" (кот, кошка, кошечка, котенок -> кот).

## Результаты других алгоритмов

Если решается задача классификации, можно сначала решить вспомогательную задачу кластеризации, и в качестве признака в первоначальной задаче взять кластер объекта.

Это обычно происходит на основе первичного анализа данных в случае, когда объекты хорошо кластеризуются.

## Агрегированные признаки

Признаки, которые агрегируют признаки некоторого объекта, тем самым также сокращая размерность признакового описания.

Полезно в задачах, в которых один объект содержит несколько однотипных параметров.

Например, человек, имеющий несколько автомобилей разной стоимости. В данном случае можно рассмотреть признаки, соответствующие максимальной/минимальной/средней стоимости автомобиля этого человека.

## *Добавление новых признаков

Чтобы эффективно решить задачу, необходимо быть экспертом в конкретной области и понимать, что влияет на конкретную целевую переменную. 

## Рекомендации

- Избегайте редко используемых дискретных значений признаков.
    - Хорошие значения характеристик должны появляться в наборе данных более 5 раз.
    - И наоборот, если значение появляется только один раз или очень редко, модель не может делать прогнозы на основе этого значения признака.
- Стремитесь к ясным и очевидным значениям.
    - Каждый признак должен иметь ясное и очевидное значение для всех участников проекта. 
- Учитывайте нестабильность входящего потока.
    - Определение признака не должно менять со временем.
    - Но при получении значения с помощью другой модели сопряжено с дополнительными трудностями.
- Чаще всего необходимо избавиться от дублирующих строк.